# Schoology Grades Investigation

The output from the following request is saved in `grades.json`. The cell below loads this file into object `raw_grades`.

```none
GET /v1/users/100032898/grades HTTP/1.1
Host: api.schoology.com
```

In [20]:
file = "grades.json"
raw_grades = dict()

with open(file,) as f:
    raw_grades = json.load(f)

The following screenshots were captured at the same time as the payload above, showing fictional student Mary Archer's grades in two sections: Algebra-I and English.

**Mary Archer's Algebra-I grades**

![Mary Archer's Algebra grades](mary-archer-grades-algebra.jpg)

**Mary Archer's English grades**

![Mary Archer's English grades](mary-archer-grades-english.jpg)

Now let's parse the payload from JSON into Python and see if we can find those interim and final grades.

In [17]:
import json
import pandas as pd
from IPython.display import display, Markdown

for section in raw_grades["section"]:
    section_id = section["section_id"]
    final_grade = section["final_grade"]

    display(Markdown(f"## Section {section_id}"))

    for grade in final_grade:
        display(Markdown(f"### Period: {grade['period_id']}"))
        display(grade)

## Section 2941242697

### Period: p822639

{'period_id': 'p822639',
 'grade': 1,
 'weight': 100,
 'comment': '',
 'scale_id': 0,
 'grading_category': [{'category_id': 38619916, 'grade': 1}]}

### Period: final

{'period_id': 'final',
 'grade': 1,
 'comment': '',
 'comment_status': None,
 'scale_id': 0}

...

{'period_id': 'a2979142114',
 'grade': 9,
 'weight': 50,
 'comment': '',
 'scale_id': 0}

### Period: p822639

{'period_id': 'p822639',
 'grade': 91.33,
 'weight': 50,
 'comment': '',
 'scale_id': 0,
 'grading_category': [{'category_id': 38619838, 'grade': 91.33}]}

### Period: final

{'period_id': 'final',
 'grade': 83.17,
 'comment': '',
 'comment_status': None,
 'scale_id': 0}

In each case, we have a period called "final". The grades for those "final periods" are 1 and 83.17%, matching the screenshots. Now, the UDM envisions both "current" and "final" grades. What is the difference? "Current" is the grade during the grading period, "Final" is a grade for a completed grading period. How can we distinguish? In Schoology there is no "term" concept, but there are grading periods. A section can be in multiple grading periods, as we see above: The first section is in two grading periods and the second section is in one. Can we cross-check the grading periods, and assume that a section is "finished" (final) if all grading periods are complete?

Note: if all grading periods are closed, then the section won't show in our Extractor's section file, because of this API setting that we're not using:

> "`include_past`: Set this to 1 to include sections from expired/past grading periods"

But, what happens to student grades for such a section? I took the Algebra-I section and temporarily changed its grading period to spring 2020 (this notebook is being written in autumn 2020). Then I changed the homework assignments to be in that same grading period, and changed the Grade Setup to keep the weighting: 50% of final grade from homework done in the spring 2020 grading period and 50% from the mid-term. Now the grade list has:

In [23]:
file = "grades-after-grading-period-change.json"
new_response = dict()
with open(file,) as f:
    new_response = json.load(f)

for section in new_response["section"]:
    section_id = section["section_id"]
    final_grade = section["final_grade"]

    display(Markdown(f"## Section {section_id}"))

    for grade in final_grade:
        display(Markdown(f"### Period: {grade['period_id']}"))
        display(grade)

## Section 2941242697

### Period: p822639

{'period_id': 'p822639',
 'grade': 1,
 'weight': 100,
 'comment': '',
 'scale_id': 0,
 'grading_category': [{'category_id': 38619916, 'grade': 1}]}

### Period: final

{'period_id': 'final',
 'grade': 1,
 'comment': '',
 'comment_status': None,
 'scale_id': 0}

...

{'period_id': 'a2979142114',
 'grade': 9,
 'weight': 50,
 'comment': '',
 'scale_id': 0}

### Period: p825791

{'period_id': 'p825791',
 'grade': 89.5,
 'weight': 50,
 'comment': '',
 'scale_id': 0,
 'grading_category': [{'category_id': 38619838, 'grade': 89.5}]}

### Period: final

{'period_id': 'final',
 'grade': 82.25,
 'comment': '',
 'comment_status': None,
 'scale_id': 0}

The final grade is now 82.25. Why is this changed? The mid-term didn't change. The homework in grading period p825791 now as a grade of 89.5, which is lower than before. **This is because I missed updating one homework assignment**. So it was user error. If I had gotten that right, then the final grade would not have changed.

Problem: how do we know it is a final grade, and not a current grade?

Answer: because the section is no longer in the section list.

Turn this into an algorithm, in psuedo code:

```
for student in Users[type=="Student"]:
    grades = get_grades(student)

    for section in grades.section:
        grade = section.grades[period=="final"]
        if section.section_id in active_sections:
            grade.status = "current"
        elif
            grade.status = "final"
```

While the grades are written out nested under a section directory, this is a special case: other section data are not written after a section falls out of an active grading period. This grades file, therefore, needs to be created outside of the normal loop through sections, which is used for building assignments, attendance events, submissions, and user activities files.

Also of note, the API call has a few options for limiting results.

> View a list of grades for the given user. The following query strings can (optionally) be appended to the path to filter results:
> * `section_id`: Specifying a section id limits results to that section.
> * `timestamp`: Specifying a timestamp limits results to recorded grades that have been updated since the given timestamp, according to the server time.
> * `grading_period_ids`: Specifying a comma-delimited list of grading period ids limits results to those grading periods. Use 'other' for the 'Other' grading period, and 'final' for final grades.

We might consider limiting the data in some way in the future, for instance by timestamp. We could limit by section, but then we need a separate process to detect the finished sections and query for their data. We can revisit that if we have a performance problem with too much data coming back for an individual student.